In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load Packages

In [18]:
!pip install warmup_scheduler
!pip install monai
!pip install -U "python-gdcm" pydicom pylibjpeg
!pip install -U torchvision
!pip install opencv-python

In [19]:
# Libraries
import os
import re
import gc
import cv2
import wandb
import PIL
from PIL import Image
from sklearn.metrics import classification_report
import random
import math
import shutil
from glob import glob
from tqdm import tqdm
from pprint import pprint
from time import time
import warnings
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle
from IPython.display import display_html
plt.rcParams.update({'font.size': 16})

from pathlib import Path

# .dcm handling
import pydicom
import nibabel as nib
from pydicom.pixel_data_handlers.util import apply_voi_lut

# Environment check
warnings.filterwarnings("ignore")

# set seaborn theme
sns.set_theme(style="darkgrid")

In [20]:
# PyTorch
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import torchvision 
import torchvision.transforms as transforms
from warmup_scheduler import GradualWarmupScheduler
import albumentations

from sklearn.model_selection import GroupKFold, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix

# MONAI 3D
from monai.transforms import Randomizable, apply_transform
from monai.transforms import Compose, Resize, ScaleIntensity, ToTensor, RandAffine
from monai.networks.nets import densenet

### Helper Function

In [21]:
def read_data():
    '''Reads in all .csv files.'''
    
    train = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    train_bbox = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
    test = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")
    ss = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv")
    
    return train, train_bbox, test, ss

def get_csv_info(csv, name="Default"):
    '''Prints main information for the speciffied .csv file.'''
    
    print(f"=== {name} ===")
    print(f"Shape:", csv.shape)
    print(f"Missing Values:", csv.isna().sum().sum(), "total missing datapoints.")
    print("Columns:", list(csv.columns), "\n")
    
    display_html(csv.head())
    print("\n")
    
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

# some patients have reverse order for the CT scan, so have a function to check
def check_reverse_required(path):
    paths = list(path.glob('*'))
    paths.sort(key=lambda x:int(x.stem))
    z_first = pydicom.dcmread(paths[0]).get("ImagePositionPatient")[-1]
    z_last = pydicom.dcmread(paths[-1]).get("ImagePositionPatient")[-1]
    if z_last < z_first:
        return False
    return True

paths = {
    'train': Path('../input/rsna-2022-cervical-spine-fracture-detection/train.csv'),
    'train_bbox': Path('../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv'),
    'train_images': Path('../input/rsna-2022-cervical-spine-fracture-detection/train_images'),
    'train_nifti_segments': Path('../input/rsna-2022-cervical-spine-fracture-detection/segmentations'),
    'test_df': Path('../input/rsna-2022-cervical-spine-fracture-detection/test.csv'),
    'test_images': Path('../input/rsna-2022-cervical-spine-fracture-detection/test_images')
}

# === 🐝 W&B ===
def save_dataset_artifact(run_name, artifact_name, path, data_type="dataset"):
    '''Saves dataset to W&B Artifactory.
    run_name: name of the experiment
    artifact_name: under what name should the dataset be stored
    path: path to the dataset'''
    
    run = wandb.init(project='RSNA_SpineFructure', 
                     name=run_name, 
                     config=CONFIG)
    artifact = wandb.Artifact(name=artifact_name, 
                              type=data_type)
    artifact.add_file(path)

    wandb.log_artifact(artifact)
    wandb.finish()
    print("Artifact has been saved successfully.")

def create_wandb_plot(x_data=None, y_data=None, x_name=None, y_name=None, title=None, log=None, plot="line"):
    '''Create and save lineplot/barplot in W&B Environment.
    x_data & y_data: Pandas Series containing x & y data
    x_name & y_name: strings containing axis names
    title: title of the graph
    log: string containing name of log'''
    
    data = [[label, val] for (label, val) in zip(x_data, y_data)]
    table = wandb.Table(data=data, columns = [x_name, y_name])
    
    if plot == "line":
        wandb.log({log : wandb.plot.line(table, x_name, y_name, title=title)})
    elif plot == "bar":
        wandb.log({log : wandb.plot.bar(table, x_name, y_name, title=title)})
    elif plot == "scatter":
        wandb.log({log : wandb.plot.scatter(table, x_name, y_name, title=title)})
        
def create_wandb_hist(x_data=None, x_name=None, title=None, log=None):
    '''Create and save histogram in W&B Environment.
    x_data: Pandas Series containing x values
    x_name: strings containing axis name
    title: title of the graph
    log: string containing name of log'''
    
    data = [[x] for x in x_data]
    table = wandb.Table(data=data, columns=[x_name])
    wandb.log({log : wandb.plot.histogram(table, x_name, title=title)})

In [22]:
# custom weighted loss function
# From: https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detect-pytorch-densenet-train#2.-Data-Split
def get_custom_loss(logits, targets):
    
    # Compute the weights
    weights = targets * competition_weights['+'] + (1 - targets) * competition_weights['-']
    
    # Losses on label and exam level
    L = torch.zeros(targets.shape, device=DEVICE)

    w = weights
    y = targets
    p = logits
    eps=1e-8

    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            L[i, j] = -w[i, j] * (
                y[i, j] * math.log(p[i, j] + eps) +
                (1 - y[i, j]) * math.log(1 - p[i, j] + eps))
            
    # Average Loss on Exam (or patient)
    Exams_Loss = torch.div(torch.sum(L, dim=1), torch.sum(w, dim=1))
    
    return Exams_Loss

## Set up parameters

In [23]:
# Environment check
warnings.filterwarnings("ignore")
# os.environ["WANDB_SILENT"] = "true"
CONFIG = {'competition': 'RSNA_SpineFracture', '_wandb_kernel': 'aot'}

# set seed
set_seed(0)

# set GPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", DEVICE)

# Kaggle Notebook Setup
DF_SIZE = 0.01
N_SPLITS = 5
KERNEL_TYPE = 'densenet121_baseline'
IMG_RESIZE = 100
STACK_RESIZE = 50
use_amp = False
NUM_WORKERS = 1
BATCH_SIZE = 2
LR = 0.05
OUT_DIM = 8
EPOCHS = 2

target_cols = ['C1', 'C2', 'C3', 
               'C4', 'C5', 'C6', 'C7',
               'patient_overall']

competition_weights = {
    '-' : torch.tensor([1, 1, 1, 1, 1, 1, 1, 7], dtype=torch.float, device=DEVICE),
    '+' : torch.tensor([2, 2, 2, 2, 2, 2, 2, 14], dtype=torch.float, device=DEVICE),
}

Device: cpu


## Load Data

In [24]:
np.random.seed(0)

df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")

# reverse the slices
df['segment_path'] = df['StudyInstanceUID'].map(lambda x: paths['train_images']/x)
df['reverse_required'] = df['segment_path'].map(check_reverse_required)

df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,segment_path,reverse_required
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,False
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,True
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,False
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,../input/rsna-2022-cervical-spine-fracture-det...,False
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,../input/rsna-2022-cervical-spine-fracture-det...,True


In [25]:
# Sample down df
instances = df.StudyInstanceUID.unique().tolist()
instances = random.sample(instances, k=int(len(instances)*DF_SIZE))
df = df[df["StudyInstanceUID"].isin(instances)].reset_index(drop=True)
print("Dataframe size:", df.shape)

# Create folds
kfold = GroupKFold(n_splits=N_SPLITS)
df['fold'] = -1

# Append fold
for k, (_, valid_i) in enumerate(kfold.split(df,
                                             groups=df.StudyInstanceUID)):
    df.loc[valid_i, 'fold'] = k
    
print("K Folds Count:")
df["fold"].value_counts(

SyntaxError: unexpected EOF while parsing (1181204886.py, line 17)

## Stack all scans together

In [ ]:
# take a look at an example
study_paths = glob(f"../input/rsna-fracture-detection/zip_png_images/1.2.826.0.1.3680043.27262/*")
study_paths.sort(key=natural_keys)
        
# Load images
#OpenCV use BGR format so we need to transfer RGB to BRG
study_images = [cv2.imread(path)[:,:,::-1] for path in study_paths]

print("# of scans:", len(study_images))
print("dimension of each scan:", study_images[0].shape)

# stack images
stacked_image = np.stack([img.astype(np.float32) for img in study_images], 
                                 axis=2).transpose(3,0,1,2)

print("dimension of each scan after stack:", stacked_image.shape)

#show the stacked image 
# cv2.imshow('image', stacked_image)

In [ ]:
class RSNADataset(Dataset, Randomizable):
    
    def __init__(self, csv, mode, transform=None):
        self.csv = csv
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return self.csv.shape[0]
    
    def randomize(self) -> None:
        '''-> None is a type annotation for the function that states 
        that this function returns None.'''
        
        MAX_SEED = np.iinfo(np.uint32).max + 1
        self.seed = self.R.randint(MAX_SEED, dtype="uint32")
        
    def __getitem__(self, index):
        # Set Random Seed
        self.randomize()
        
        dt = self.csv.iloc[index, :]
        study_paths = glob(f"../input/rsna-fracture-detection/zip_png_images/{dt.StudyInstanceUID}/*")
        study_paths.sort(key=natural_keys)
        
        # Load images
        study_images = [cv2.imread(path)[:,:,::-1] for path in study_paths]
        # Stack all scans into 1
        stacked_image = np.stack([img.astype(np.float32) for img in study_images], 
                                 axis=2).transpose(3,0,1,2)
        
        if self.transform:
            if isinstance(self.transform, Randomizable):
                self.transform.set_random_state(seed=self.seed)
                
            stacked_image = apply_transform(self.transform, stacked_image)
        
        if self.mode=="test":
            return {"image": stacked_image}
        else:
            targets = torch.tensor(dt[target_cols]).float()
            return {"image": stacked_image,
                    "targets": targets}

In [ ]:
# send the data to GPU
def data_to_device(data):
    
    image, targets = data.values()
    return image.to(DEVICE), targets.to(DEVICE)

In [ ]:
# transform
train_transforms = Compose([ScaleIntensity(), 
                            Resize((IMG_RESIZE, IMG_RESIZE, STACK_RESIZE)), 
                            # TODO - add more here
                            ToTensor()])
valid_transforms = Compose([ScaleIntensity(), 
                          Resize((IMG_RESIZE, IMG_RESIZE, STACK_RESIZE)), 
                          ToTensor()])

In [ ]:
# Sanity Check

# Sample data
sample_df = df.head(6)

# Instantiate Dataset object
dataset = RSNADataset(csv=sample_df, mode="train", transform=train_transforms)
# The Dataloader
dataloader = DataLoader(dataset, batch_size=3, shuffle=False)

# Output of the Dataloader
for k, data in enumerate(dataloader):
    image, targets = data_to_device(data)
    print( f"Batch: {k}", "\n" +
          "Image:", image.shape, "\n" +
          "Targets:", targets, "\n" +
          "="*50)

In [ ]:
del dataset, dataloader, image, targets

# gabage collector which is used to free up memory, return the counts of objects 

gc.collect()

## Loss & Gradual warmup

Reference link: [HERE](https://stackoverflow.com/questions/42479902/what-does-view-do-in-pytorch)

torch.view(-1):
* view() reshapes the tensor without copying memory, similar to numpy's reshape().
* -1 flatten the tensor

In [ ]:
CRITERION = nn.BCEWithLogitsLoss(reduction='none')

def get_criterion(logits, target): 
    loss = CRITERION(logits.view(-1), target.view(-1))
    return loss

In [ ]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    '''
    src: https://www.kaggle.com/code/boliu0/monai-3d-cnn-training/notebook
    '''
    
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, 
                                                       total_epoch, after_scheduler)
    
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier 
                                                     for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) 
                    for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) 
                    for base_lr in self.base_lrs]

## Log the info

In [ ]:
def add_in_file(text, f):
    
    with open(f'log_{KERNEL_TYPE}.txt', 'a+') as f:
        print(text, file=f)

## Train Epoch

In [ ]:
def train_epoch(model, dataloader, optimizer, epoch, f):
    
    # Add info to file
    print("Training...")
    add_in_file('Training...', f)
    
    # Track training time for 1 epoch
    start_time = time()
    
    # === TRAIN ===
    model.train()
    train_losses, train_comp_losses = [], []
    
    # Loop through the data
    bar = tqdm(dataloader)
    for data in bar:
        image, targets = data_to_device(data)
        
        # Train & Optimize
        optimizer.zero_grad()
        logits = model(image)
        loss = get_criterion(logits, targets)
        loss.sum().backward()
        optimizer.step()
        
        # === COMP LOSS ===
        comp_loss = get_custom_loss(logits, targets)

        # Save losses
        train_losses.append(loss.detach().cpu().numpy())
        train_comp_losses.append(comp_loss.detach().cpu().numpy().mean())
        
        gc.collect()

    # Compute Overall Loss
    mean_train_loss = np.mean(train_losses)
    mean_comp_loss = np.mean(train_comp_losses)
    
    # Save info
    total_time = round((time() - start_time)/60, 3)
    add_in_file('Train Mean Loss: {}'.format(mean_train_loss), f)
    add_in_file('Train Mean Comp Loss: {}'.format(mean_comp_loss), f)
    add_in_file('~~~ Train Time: {} mins ~~~'.format(total_time), f)
    
    # 🐝 Log to W&B
    wandb.log({"train_loss": mean_train_loss,
               "train_comp_loss": mean_comp_loss,}, step=epoch)
                
    # Print info
    print("Train Mean Loss:", mean_train_loss)
    print("Train Mean Comp Loss:", mean_comp_loss)
    print(f"~~~ Train Time: {total_time} mins ~~~")
    
    return mean_train_loss

## Validation Epoch

In [ ]:
def valid_epoch(model, dataloader, epoch, f):
    
    # Add info to file
    print("Validation...")
    add_in_file('Validation...', f)
    
    # Track validation time for 1 epoch
    start_time = time()
    
    # === EVAL ===
    model.eval()
    valid_preds, valid_targets, valid_comp_loss = [], [], []
    
    with torch.no_grad():
        for data in dataloader:
            
            image, targets = data_to_device(data)
            logits = model(image)
            
            # === COMP LOSS ===
            comp_loss = get_custom_loss(logits, targets)
            # Save actuals, preds and losses
            valid_targets.append(targets.detach().cpu())
            valid_preds.append(logits.detach().cpu())
            valid_comp_loss.append(comp_loss.detach().cpu().numpy().mean())
            
            gc.collect()
            
    # Overall Valid Loss
    valid_losses = get_criterion(torch.cat(valid_preds), torch.cat(valid_targets)).numpy()
    mean_valid_loss = np.mean(valid_losses)
    
    # Overall Competition Loss
    mean_comp_valid_loss = np.mean(valid_comp_loss)
    
    # Compute Area Under Curve
    PREDS = np.concatenate(torch.cat(valid_preds).numpy())
    TARGETS = np.concatenate(torch.cat(valid_targets).numpy())
    auc = roc_auc_score(TARGETS, PREDS)
    
    # Save info
    total_time = round((time() - start_time)/60, 3)
    add_in_file('Valid Mean Loss: {}'.format(mean_valid_loss), f)
    add_in_file('Valid Mean Comp Loss: {}'.format(mean_comp_valid_loss), f)
    add_in_file('Valid AUC: {}'.format(auc), f)
    add_in_file('~~~ Valid Time: {} mins ~~~'.format(total_time), f)
    
    # 🐝 Log to W&B
    wandb.log({"valid_loss": mean_valid_loss,
               "valid_comp_loss": mean_comp_valid_loss,
               "valid_auc": auc}, step=epoch)
        
    # Print info
    print("Valid Mean Loss:", mean_valid_loss)
    print("Valid Mean Comp Loss:", mean_comp_valid_loss)
    print("Valid AUC:", auc)
    print(f"~~~ Validation Time: {total_time} mins ~~~")
    
    return mean_valid_loss

In [ ]:
def run_train(fold):
    
    # 🐝 W&B Tracking
    RUN_CONFIG = CONFIG.copy()
    params = dict(model="densenet121", 
                  epochs=EPOCHS, 
                  split=N_SPLITS, 
                  batch=BATCH_SIZE, lr=LR,
                  img_size=IMG_RESIZE, stack_size=STACK_RESIZE,
                  data_size=DF_SIZE)
    RUN_CONFIG.update(params)
    run = wandb.init(project='RSNA_SpineFracture', config=CONFIG)
    
    # Get the train and valid data
    train = df[df["fold"] != fold].reset_index(drop=True)
    valid = df[df["fold"] == fold].reset_index(drop=True)
    
    # Create the Dataset & Dataloader
    train_dataset = RSNADataset(csv=train, mode="train", 
                                transform=train_transforms)
    valid_dataset = RSNADataset(csv=valid, mode="train", 
                                transform=valid_transforms)
    trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                             sampler=RandomSampler(train_dataset))
    validloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)
    
    # Model
    model = densenet.densenet121(spatial_dims=3, in_channels=3,
                                 out_channels=OUT_DIM)
    model.class_layers.out = nn.Sequential(nn.Linear(in_features=1024, out_features=OUT_DIM), 
                                           nn.Softmax(dim=1))
    model.to(DEVICE)
    wandb.watch(model, log_freq=100) # 🐝
    
    # Optimizer & Scheduler
    optimizer = optim.Adam(model.parameters(), lr=LR)
    scheduler_cosine = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 2)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, 
                                                total_epoch=1, 
                                                after_scheduler=scheduler_cosine)
    
    # Initiate initial loss
    valid_loss_BEST = 1000
    # Create model name
    model_file = f'{KERNEL_TYPE}_best_fold{fold}.pth'
    # Create file to save outputs
    f = open(f'log_{KERNEL_TYPE}.txt', 'a')
    
    for epoch in range(EPOCHS):
        
        add_in_file('======== Epoch: {}/{} ========'.format(epoch+1, EPOCHS), f)
        print("="*8, f"Epoch {epoch}", "="*8)
        
        scheduler_warmup.step(epoch-1)
        
        # Train & Validate
        mean_train_loss = train_epoch(model, trainloader, optimizer, epoch, f)
        mean_valid_loss = valid_epoch(model, validloader, epoch, f)
        
        # Save model
        if mean_valid_loss < valid_loss_BEST:
            print('Saving model ...')
            add_in_file('Saving model => {}'.format(model_file), f)
            torch.save(model.state_dict(), model_file)
            valid_loss_BEST = mean_valid_loss
            
    torch.cuda.empty_cache()
    gc.collect()
    
    # 🐝 Experiment End
    wandb.finish()

# Train

In [ ]:
run_train(fold=0)

In [ ]:
# 🐝 Save Artifacts
save_dataset_artifact(run_name="save_logs", artifact_name="logs",
                      path="../input/rsna-fracture-detection/log_densenet121_baseline.txt", data_type="dataset")
save_dataset_artifact(run_name="save_model", artifact_name="model",
                      path="../input/rsna-fracture-detection/densenet121_baseline_best_fold0.pth", data_type="model")

## Reference

* [RSNA Fracture Detect: PyTorch DenseNet train](https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detect-pytorch-densenet-train)